In [1]:
# from pycoral.adapters import classify, common
# from pycoral.utils.edgetpu import make_interpreter
# import platform
# import tflite_runtime.interpreter as tflite

# _EDGETPU_SHARED_LIB = {
#   'Linux': 'libedgetpu.so.1',
#   'Darwin': 'libedgetpu.1.dylib',
#   'Windows': 'edgetpu.dll'
# }[platform.system()]

# print(_EDGETPU_SHARED_LIB)

# interp = tflite.load_delegate(_EDGETPU_SHARED_LIB)

# print(help(interp))

In [ ]:
# from sklearn.datasets import fetch_openml
# from sklearn.linear_model import SGDClassifier
# from sklearn.model_selection import cross_val_score
# import pandas as pd
# import os


# def get_mnist784_dataset():
#     data_dir = os.path.join(os.getcwd(), "datasets")
#     if not os.path.exists(data_dir):
#         os.mkdir(data_dir)
#     mnist_dir =  os.path.join(os.getcwd(), "datasets", "mnist_784")
#     if not os.path.exists(mnist_dir):
#         os.mkdir(mnist_dir)

#     mnist = fetch_openml('mnist_784', as_frame=False)
#     data_df = pd.DataFrame(mnist.data, columns=mnist.feature_names)
#     target_df = pd.DataFrame(mnist.target, columns=mnist.target_names)
#     data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
#     target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
#     data_df.to_csv(data_path)
#     target_df.to_csv(target_path)
#     X, y = mnist.data, mnist.target
#     return X, y

# def mnist784_df_from_csv():
#     data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
#     target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
#     data_df = pd.read_csv(data_path)
#     target_df = pd.read_csv(target_path)
#     X_df, y_df = data_df, target_df
#     X_df.drop(columns="Unnamed: 0", inplace=True)
#     y_df.drop(columns="Unnamed: 0", inplace=True)
#     return X_df, y_df

# def train_predict_mnist784():
#     X_df, y_df = mnist784_df_from_csv()
#     X = X_df.values
#     y = y_df.values
#     some_digit = X[0]
#     X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
#     y_train_5 = (y_train == 5)
#     y_test_5 = (y_test == 5)
#     sgd_clf = SGDClassifier(random_state=42)
#     sgd_clf.fit(X_train, y_train_5)
#     sgd_clf.predict([some_digit])
#     scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
#     return scores

# res = train_predict_mnist784()
# print(res)

In [107]:
import tensorflow as tf


input_shape = (44, 92)
n_outputs = 130

data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal", input_shape=(44, 92, 1)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

model = tf.keras.Sequential()
model.add(data_augmentation)
# model.add(tf.keras.Input(shape=input_shape))
model.add(tf.keras.layers.Reshape(target_shape=(input_shape[0], input_shape[1], 1)))
model.add(tf.keras.layers.Conv2D(32, 3, activation="relu"))
model.add(tf.keras.layers.Conv2D(32, 3, activation="relu"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation="softmax"))
model.add(tf.keras.layers.Dense(n_outputs))

x = tf.ones((1, 44, 92, 1))
y = model(x)
model.summary()




Model: "sequential_145"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_144 (Sequential  (None, 44, 92, 1)         0         
 )                                                               
                                                                 
 reshape_19 (Reshape)        (None, 44, 92, 1)         0         
                                                                 
 conv2d_201 (Conv2D)         (None, 42, 90, 32)        320       
                                                                 
 conv2d_202 (Conv2D)         (None, 40, 88, 32)        9248      
                                                                 
 flatten_7 (Flatten)         (None, 112640)            0         
                                                                 
 dense_76 (Dense)            (None, 64)                7209024   
                                                    

In [108]:
import os
import cv2
import tensorflow as tf
import numpy as np

def normalize_img(img):
    img = img * 1.0/255
    return img

def imgs_to_dict(image_dir):
    img_dict = {}
    for fname in os.listdir(image_dir):
        img_path = os.path.join(image_dir, fname)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        name = fname.split(".")[0]
        idx = name.split("image")[-1]
        img_dict[idx] = img
    return img_dict

def images_to_arr(obj):
    imgs = []
    if isinstance(obj, dict):
        imgs = [normalize_img(img) for img in obj.values()]
    elif isinstance(obj, str):
        dir_path = None 
        if os.path.isdir(imgs):
            dir_path = imgs
        else:
            dir_path = os.path.join(os.getcwd(), obj)
        for fname in sorted(os.listdir(dir_path)):
            img_path = os.path.join(dir_path, fname)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = normalize_img(img)
            imgs.append(img)
    else:
        raise TypeError(f"type {type(obj)} is not supported")
    return np.array(imgs)

dirpath = r"C:\py_repos\image_classifier\integer_images"

d = imgs_to_dict(dirpath)
a = images_to_arr(d)
targets = np.array([int(i) for i in list(d.keys())])

x_train, x_test = a[:30], a[30:40]
y_train, y_test = np.array(targets[:30]), np.array(targets[30:40]) 
x_val, y_val = np.array(a[40:50]), np.array(targets[40:50])

model.compile(optimizer='adam',
            loss="sparse_categorical_crossentropy",
            metrics=['accuracy'])

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100)
# model.evaluate(x_val, y_val)
scores = history.history
loss = np.average(scores["loss"])
accuracy = np.average(scores["accuracy"])
print("Loss Average: {}".format(loss))
print("Accuracy Average: {}".format(accuracy))
print("Loss: {}".format(scores["loss"][-1]))
print("Accuracy: {}".format(scores["accuracy"][-1]))

Epoch 1/100
1/1 [==============================] - 1s 936ms/step - loss: 10.4089 - accuracy: 0.0000e+00 - val_loss: 14.1645 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 179ms/step - loss: 9.2325 - accuracy: 0.0333 - val_loss: 13.3057 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 183ms/step - loss: 8.8000 - accuracy: 0.0333 - val_loss: 14.0584 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 182ms/step - loss: 8.3263 - accuracy: 0.0333 - val_loss: 14.1084 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 187ms/step - loss: 8.1188 - accuracy: 0.0333 - val_loss: 15.8134 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 183ms/step - loss: 7.9307 - accuracy: 0.0333 - val_loss: 15.6016 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 179ms/step - loss: 7.7557 - accuracy: 0.0333 - val_loss: 15.3846 

In [ ]:
import tensorflow as tf


initial_model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(44, 92, 3)),
        tf.keras.layers.Conv2D(32, 5, strides=2, activation="relu"),
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = tf.keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 44, 92, 1))
features = feature_extractor(x)

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
model.compile(...)
model.fit(...)

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))